# **DeepLense: Learning Mass of Dark Matter Halo**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!tar zxf gdrive/MyDrive/task_3.tgz

In [ ]:
import numpy as np
from os import listdir
import matplotlib.pyplot as plt
import torch
from torch.nn import MSELoss, Module, Conv2d, Linear
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

%matplotlib inline

In [ ]:
files = listdir('lens_data/')

images_arr = []
halo_mass_arr = []

for f in files:
    image, mass = np.load(f"lens_data/{f}", allow_pickle=True)
    images_arr.append(image)
    halo_mass_arr.append(mass)
    
images_arr = np.stack(np.expand_dims(images_arr, axis=1)).astype(np.float32) # (n, channel, width, height)
halo_mass_arr = np.stack(np.expand_dims(halo_mass_arr, axis=1)).astype(np.float32) # (value)

In [ ]:
print(images_arr.shape)
print(halo_mass_arr.shape)

## **Displaying Lensing Images**

In [ ]:
row = 3
col = 3
figure, axis = plt.subplots(row, col, figsize= (12,12), gridspec_kw= {'wspace':0, 'hspace':0.1})
index=0

for i in range(row):
  for j in range(col):
    img = axis[i][j].imshow(images_arr[index][0], cmap='gist_gray')
    axis[i][j].set_title(f'fraction_mass: {halo_mass_arr[index][0]:.3}')
    axis[i][j].axis('off')
    index+=1
plt.show()

## **Data Augmentation**


The data set consists of 20000 black and white (single channel) 150*150 unnormalized lensing images. We need to feature scale them by standardizing (z-score normalise) during the image preprocessing.  
Also since the sample above shows that most images are centered, we will crop the image from the center.

In [ ]:
# Calculating the respective mean and standard deviation
IMG_MEAN, IMG_STD = images_arr.mean(), images_arr.std()

In [ ]:
print(IMG_STD, IMG_MEAN)

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Normalize(mean=[IMG_MEAN], std=[IMG_STD])
])

## **Data Loading**


We use a custom dataset to store the images and the labels.

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, x, y, indexes=None):
        self.x = x[indexes]
        self.y = y[indexes]

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, idx):
        image, label = self.x[idx], self.y[idx]
        
        image = torch.tensor(image).float()
        label = torch.tensor(label).float()
        
        image = preprocess(image)

        return  image, label

## **Split the dataset**

In [ ]:
n = len(images_arr)
t = int(0.9 * n)

train_indices = np.arange(0, t)
test_indices = np.arange(t, n)

In [ ]:
train_dataset = ImageDataset(images_arr, halo_mass_arr, train_indices)
test_dataset = ImageDataset(images_arr, halo_mass_arr, test_indices)

batch_size = 64

train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# **Loss Function**


We will use the Mean Squared Error as the loss function.

In [ ]:
def mse(pred, true):
  return (np.abs(pred - true)**2).mean()

In [ ]:
loss = MSELoss()

## **Creating the model**


We will use the VGG13 CNN architecture modifying only the first and last layer for our custom input (single channel) and the regression output ie. 1.

In [ ]:
class VGG13Regression(Module):
    def __init__(self, channels, op_size):
        super(VGG13Regression, self).__init__()
        self.vgg13 = models.vgg13(pretrained=True)
        self.vgg13.features[0] = Conv2d(
            in_channels=channels,
            out_channels=64,
            kernel_size=(3,3),
            stride=(2,2),
            padding=(2,2),
            bias=True
        )
        self.vgg13.classifier[6] = Linear(
            in_features=4096,
            out_features=op_size,
            bias=True
        )
    def forward(self, x):
        return self.vgg13(x)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = VGG13Regression(1,1).to(device)

In [ ]:
# A larger learning rate results in a relatively volatile model.
lr = 1e-4

num_of_epochs = 30

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

## **Training the model**

In [ ]:
losses = []
for epoch in range(num_of_epochs):
    print(f'Epoch {epoch}/{num_of_epochs - 1}')
    epoch_loss = 0.0
    steps_in_epoch = 0
    for _, (image, mass) in enumerate(train_data_loader):        
        optimizer.zero_grad()
        
        image = image.to(device)
        mass = mass.to(device)
        
        preds = model(image)
        
        b_loss = loss(preds, mass)
        
        b_loss.backward()
        
        optimizer.step()
        
        epoch_loss += b_loss
        steps_in_epoch += 1
        
    w_loss = (epoch_loss/steps_in_epoch).detach().item()
    losses.append(w_loss)
    print(f'Loss {w_loss}')

In [ ]:
plt.xlabel('Epochs')
plt.ylabel('Training Loss')
plt.title('Epochs vs Loss for Training data')
plt.rcParams["figure.figsize"] = (10,3)
plt.plot(np.array(losses))

## **Testing**

In [ ]:
predicted_mf_list = []
real_mf_list = []

for step, (image_d, fm_d) in enumerate(test_data_loader):
    optimizer.zero_grad()
    
    image_d = image_d.to(device)
    fm_d = fm_d.to(device)
    
    preds = model(image_d)
    predicted_mf_list.append(preds.cpu().detach().numpy())
    real_mf_list.append(fm_d.cpu().numpy())

predicted_mf_list = np.concatenate(predicted_mf_list)
real_mf_list = np.concatenate(real_mf_list)

In [ ]:
test_mse = mse(predicted_mf_list,real_mf_list)
print(f'Test MSE: {test_mse}')

## **Save the model**

In [ ]:
torch.save(model.state_dict(), 'ct3_model.pth')

## Generate pdf

In [ ]:
!wget -nc https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
from colab_pdf import colab_pdf
colab_pdf('/content/drive/MyDrive/Colab Notebooks/ct3.ipynb')